In [329]:
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns
import googleapiclient.errors
from pathlib import Path

In [330]:
API_KEY = 
channelids = ['UCNAf1k0yIjyGu3k9BwAg3lg','UCsNkuVj19wfclwicQ4DgAUw','UCTMR5dtI1kuZmcHazf6noWQ,UCmJf6_W7W-Q7rTd5skVEgqA','UCLBuNqpwvWe9f_-iOegwFrQ','UCsbvidDqbfMoOwydG7xF8mQ','UCqvaXJ1K3HheTPNjH-KpwXQ','UCFKE7WVJfvaHW5q283SxchA','UCAeAB8ABXGoGMbXuYPmiu2A'
             ]
                      

youtube = build('youtube','v3', developerKey = API_KEY)

In [331]:
def get_channel_stats(youtube, channelids):
    
    all_data=[]
    
    request = youtube.channels().list(
            part='snippet,contentDetails,statistics,status,topicDetails,brandingSettings',
            id=','.join (channelids))
    
        
    response = request.execute()
    
    for i in range(len(response['items'])):
    
        data = dict(Channelname = response['items'][i]['snippet']['title'],
               subs = response['items'][i]['statistics']['subscriberCount'], 
               views = response['items'][i]['statistics']['viewCount'],
               Total_videos = response['items'][i]['statistics']['videoCount'],
               playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'],
               published_At = response['items'][i]['snippet']['publishedAt'],
               topic_ids = response['items'][i]['topicDetails']['topicCategories'],
               description= response['items'][i]['brandingSettings']['channel']['description'])
                                                                
        all_data.append(data)
    return all_data

In [332]:
Channel_Stats= get_channel_stats(youtube, channelids)

In [380]:
channel_data = pd.DataFrame(Channel_Stats)

channel_data['subs'] = pd.to_numeric(channel_data['subs'])
channel_data['views'] = pd.to_numeric(channel_data['views'])
channel_data['Total_videos'] = pd.to_numeric(channel_data['Total_videos'])



In [338]:

channel_data.to_csv('channel_Data_Final.csv')


In [417]:
playlist_id = channel_data.loc[channel_data['Channelname'] == 'The Swedish Investor','playlist_id'].iloc[0]

playlist_id



'UUAeAB8ABXGoGMbXuYPmiu2A'

In [418]:
## function for video ids

def get_video_ids(youtube, playlist_id):
    
    request = youtube.playlistItems().list(
                part = 'contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    video_ids = []
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        next_page_token = response.get('nextPageToken')
        more_pages = True

    while more_pages:
        if next_page_token is None:
                more_pages = False
    
        else:
            request = youtube.playlistItems().list(
                part = 'contentDetails',
                playlistId = playlist_id,
                maxResults = 50,
                pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
                                  
            next_page_token = response.get('nextPageToken') 
    
    return (video_ids)

In [419]:
video_ids = get_video_ids(youtube, playlist_id)



In [420]:
def get_video_Details(youtube,video_ids):
    all_video_stats =[]
    
    
    for i in range(0,len(video_ids),50):
        request = youtube.videos().list(
            part = 'snippet,statistics,status,topicDetails,recordingDetails,contentDetails,liveStreamingDetails',
            id = ','.join(video_ids[i:i+50]))
        response= request.execute()
                   
        for video in response['items']:
            
            video_stats = dict(Title =video['snippet']['title'],
                               Published_date = video['snippet']['publishedAt'],
                               Views = video['statistics']['viewCount'],
                               Likes = video['statistics']['likeCount'],
                               comments = video['statistics']['commentCount'],
                               livebroadcastcontent=video['snippet']['liveBroadcastContent'],
                               duration =video['contentDetails']['duration'],
                               madeforkids =video['status']['madeForKids']
                              )
            all_video_stats.append(video_stats)
        
        return all_video_stats
   

In [421]:
video_details = get_video_Details(youtube,video_ids)

video_data = pd.DataFrame(video_details)

video_data['Published_date'] = pd.to_datetime(video_data['Published_date']).dt.date
video_data['Views'] = pd.to_numeric(video_data['Views'])
video_data ['Month'] = pd.to_datetime(video_data ['Published_date']).dt.strftime('%b')
video_data['Likes'] = pd.to_numeric(video_data['Likes'])


In [422]:
video_data

,Title,Published_date,Views,Likes,comments,livebroadcastcontent,duration,madeforkids,Month
0,"How Benjamin Graham Made His First $1,000,000",2023-04-29,53918,1820,65,none,PT33M26S,False,Apr
1,Why You Can Beat the Investment Professionals ...,2023-04-14,59790,2053,125,none,PT16M41S,False,Apr
2,Why Some Stocks are Cheap & Others Expensive,2023-03-20,51320,1989,155,none,PT13M32S,False,Mar
3,"RICHER, WISER, HAPPIER (BY WILLIAM GREEN)",2023-01-30,103386,3265,127,none,PT15M19S,False,Jan
4,Value Investing Explained in 5 Levels of Diffi...,2022-12-19,137055,4760,164,none,PT18M36S,False,Dec
5,Warren Buffett: 12 Mistakes Every Investor Makes,2022-10-29,401840,9945,570,none,PT24M40S,False,Oct
6,The Story of the Great Bear of Wall Street,2022-10-22,198593,4744,148,none,PT28M19S,False,Oct
7,THE AUTOMATIC MILLIONAIRE SUMMARY (BY DAVID BACH),2022-10-01,109711,4035,244,none,PT12M6S,False,Oct
8,Warren Buffett: How to Identify Superior Stock...,2022-07-29,79212,2596,189,none,PT12M56S,False,Jul
9,Bitcoin: A Long-Term Buy?,2022-06-24,122151,4572,891,none,PT24M38S,False,Jun


In [423]:
video_data.to_csv('The Swedish Investor.csv')